In [97]:
# Dependencies
import numpy as np
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, MetaData, Table, func

In [44]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [45]:
inspector = inspect(engine)

In [46]:
columns = inspector.get_columns('Measurement')
for x in columns:
    print(x)

{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'station', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'date', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'prcp', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'tobs', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}


In [47]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [54]:
session.query(func.count(Measurement.tobs)).all()

[(19550)]

## D1: Determine the Summary Statistics for June

In [98]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 
june = session.query(Measurement.date, Measurement.tobs).filter(extract('month', Measurement.date) == 6).all()

In [99]:
#  3. Convert the June temperatures to a list.
june_ls = list(june)

In [100]:
# 4. Create a DataFrame from the list of temperatures for the month of June. 
june_df = pd.DataFrame(june_ls)

In [101]:
# 5. Calculate and print out the summary statistics for the June temperature DataFrame.
june_df.describe()

,tobs
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


## D2: Determine the Summary Statistics for December

In [79]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.
dec = session.query(Measurement.date, Measurement.tobs).filter(extract('month', Measurement.date) == 12).all()

In [81]:
# 7. Convert the December temperatures to a list.
dec_ls = list(dec)

In [82]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 
dec_df = pd.DataFrame(dec_ls)

In [84]:
# 9. Calculate and print out the summary statistics for the Decemeber temperature DataFrame.
dec_df.describe()

,tobs
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


## D3: Additional Queries 

In [106]:
station_june = session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station)).filter(extract('month', Measurement.date) == 6).all()
station_june

[('USC00518838', 38),
 ('USC00517948', 148),
 ('USC00511918', 165),
 ('USC00514830', 186),
 ('USC00516128', 223),
 ('USC00519523', 232),
 ('USC00513117', 236),
 ('USC00519281', 236),
 ('USC00519397', 236)]

In [118]:
station_dec = session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station)).filter(extract('month', Measurement.date) == 12).all()
station_dec

[('USC00518838', 19),
 ('USC00517948', 109),
 ('USC00511918', 152),
 ('USC00514830', 178),
 ('USC00516128', 206),
 ('USC00519397', 210),
 ('USC00519523', 210),
 ('USC00513117', 216),
 ('USC00519281', 217)]

In [128]:
station_june_min = session.query(Measurement.station, func.min(Measurement.tobs)).\
group_by(Measurement.station).order_by(func.min(Measurement.tobs)).filter(extract('month', Measurement.date) == 6).all()
station_june_max
station_june_min

[('USC00517948', 64.0),
 ('USC00519281', 65.0),
 ('USC00511918', 66.0),
 ('USC00516128', 66.0),
 ('USC00514830', 67.0),
 ('USC00518838', 67.0),
 ('USC00513117', 68.0),
 ('USC00519397', 70.0),
 ('USC00519523', 71.0)]

In [126]:
station_june_max = session.query(Measurement.station, func.max(Measurement.tobs)).\
group_by(Measurement.station).order_by(func.max(Measurement.tobs)).filter(extract('month', Measurement.date) == 6).all()
station_june_max

[('USC00518838', 78.0),
 ('USC00516128', 79.0),
 ('USC00514830', 81.0),
 ('USC00517948', 82.0),
 ('USC00519281', 82.0),
 ('USC00519523', 82.0),
 ('USC00513117', 83.0),
 ('USC00519397', 84.0),
 ('USC00511918', 85.0)]